In [ ]:
import torch
import whisper
import pyaudio
import wave
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import os
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

from rapidfuzz import process

intent_labels = ["MOVE_FORWARD", "TURN_LEFT", "STOP", "TURN_RIGHT"]

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
model.load_state_dict(torch.load("models/intent_model.pt", map_location=torch.device('cpu')))
model.eval()

command_phrases = ["move forward", "turn left", "stop", "turn right"]

def correct_command(text):
    best_match, score, _ = process.extractOne(text.lower(), command_phrases)
    if score > 70:
        return best_match
    return text.lower()

def predict_intent(text):
    corrected_text = correct_command(text)
    inputs = tokenizer(corrected_text, return_tensors="pt", truncation=True, padding=True, max_length=32)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted = torch.argmax(outputs.logits, dim=1)
    return intent_labels[predicted.item()]

# Example usage:
recognized_text = "ton right ton ight"
intent = predict_intent(recognized_text)
print("Detected intent:", intent)

Correct intent code to be used with recognising words if not pronounced properly.